In [ ]:
%pip install wandb pytorch_lightning rouge scikit-learn sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import pytorch_lightning as pl
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import EarlyStoppingCallback
from transformers import get_linear_schedule_with_warmup
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import wandb
from rouge import Rouge
import random

# WandB 설정
wandb.init(project="mbart50-T5_ko_v6")

# mBART50 모델과 tokenizer 로드 (한국어-영어 번역용)
tokenizer_ko_en = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer_ko_en.src_lang = 'ko_KR'
tokenizer_ko_en.tgt_lang = 'en_XX'
model_ko_en = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# mBART50 모델과 tokenizer 로드 (영어-한국어 번역용)
tokenizer_en_ko = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer_en_ko.src_lang = 'en_XX'
tokenizer_en_ko.tgt_lang = 'ko_KR'
model_en_ko = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# T5 모델과 tokenizer 로드 (요약용)
tokenizer_summary = T5Tokenizer.from_pretrained("t5-base")
model_summary = T5ForConditionalGeneration.from_pretrained("t5-base")

# 데이터 증강 및 전처리 함수 정의
def augment_text(text):
    words = text.split()
    if len(words) > 3:
        random.shuffle(words)
    return ' '.join(words)

# 데이터 로드 및 전처리
class MultiTaskDataset(Dataset):
    def __init__(self, data, tokenizer, max_len, augment=False, is_test=False):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.augment = augment
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        dialogue = self.data.iloc[index]['dialogue']
        topic = self.data.iloc[index]['topic']
        dialogue_length = len(dialogue.split())

        # 대화 길이에 따른 max_length 조정
        if dialogue_length > 100:
            max_len = 512
        elif dialogue_length > 50:
            max_len = 256
        else:
            max_len = 128

        # 데이터 증강 적용 (특정 주제일 때만 적용)
        if self.augment and topic in ['운동하기', '건강한 음식']:
            dialogue = augment_text(dialogue)

        inputs = self.tokenizer(dialogue, max_length=max_len, truncation=True, padding='max_length', return_tensors="pt")

        if not self.is_test:
            summary = self.data.iloc[index]['summary']
            labels = self.tokenizer(summary, max_length=self.max_len, truncation=True, padding='max_length', return_tensors="pt").input_ids
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': labels.squeeze()
            }
        else:
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze()
            }

# 데이터 로드
train_df = pd.read_csv('/content/drive/MyDrive/AI_Lab/AI_Lab_online/basic/NLP/Competition/Code/data/train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/AI_Lab/AI_Lab_online/basic/NLP/Competition/Code/data/dev.csv')

train_data = MultiTaskDataset(train_df, tokenizer_ko_en, max_len=512, augment=True)
val_data = MultiTaskDataset(val_df, tokenizer_ko_en, max_len=512, augment=False)
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=False)

# ROUGE 평가 함수 수정 (한국어 번역 후 요약 결과 평가)
def compute_metrics(pred):
    rouge = Rouge()
    preds = pred.predictions
    labels = pred.label_ids

    # Step 1: 영어 요약을 한국어로 번역
    decoded_preds_en = tokenizer_ko_en.batch_decode(preds, skip_special_tokens=True)
    translated_preds_ko = []
    for pred in decoded_preds_en:
        inputs = tokenizer_en_ko(pred, return_tensors="pt", padding="max_length", max_length=512, truncation=True)
        inputs = inputs.input_ids.to(trainer.model.device)
        translated_output = model_en_ko.generate(inputs, num_beams=4, max_length=512)
        translated_pred = tokenizer_en_ko.decode(translated_output[0], skip_special_tokens=True)
        translated_preds_ko.append(translated_pred)

    # Step 2: 한국어 번역된 요약과 실제 레이블(한국어 요약) 비교
    decoded_labels = tokenizer_ko_en.batch_decode(labels, skip_special_tokens=True)

    # Step 3: 한국어 번역된 예측 결과와 실제 레이블을 기반으로 ROUGE 계산
    rouge_score = rouge.get_scores(translated_preds_ko, decoded_labels, avg=True)
    return {
        'rouge-1': rouge_score['rouge-1']['f'],
        'rouge-2': rouge_score['rouge-2']['f'],
        'rouge-l': rouge_score['rouge-l']['f'],
    }

# 모델 정의
class MultiTaskModel(pl.LightningModule):
    def __init__(self, model, tokenizer, lr, num_warmup_steps, num_training_steps):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.lr = lr
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps

    def forward(self, input_ids, attention_mask=None, labels=None):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    def training_step(self, batch, batch_idx):
        outputs = self.model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        loss = outputs.loss
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

# Trainer 설정
training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/AI_Lab/AI_Lab_online/basic/NLP/Competition/Code/results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    fp16=True,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=15,
    predict_with_generate=True,
    logging_dir='/content/drive/MyDrive/logs',
    load_best_model_at_end=True,
    report_to="wandb",
)

# EarlyStoppingCallback 추가
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

# Trainer 객체 생성
trainer = Seq2SeqTrainer(
    model=model_ko_en,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer_ko_en,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)




<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
torch.cuda.empty_cache()
trainer.train()


In [ ]:
# 추론 함수 수정 (sample_submission.csv 형식으로 파일 생성)
def inference_with_translation(trainer, tokenizer_en, tokenizer_ko, test_dataloader, output_file_path, test_data_ids):
    trainer.model.eval()
    preds = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainer.model.to(device)

    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Step 1: 한국어 대화 입력 -> 영어 요약 출력 (요약 단계)
        outputs = trainer.model.generate(input_ids=input_ids, attention_mask=attention_mask, num_beams=4, max_length=512)

        # Step 2: 영어 요약을 다시 디코딩
        decoded_preds = [tokenizer_en.decode(output, skip_special_tokens=True) for output in outputs]

        # Step 3: 영어 요약 -> 한국어로 다시 번역 (번역 단계)
        for pred in decoded_preds:
            inputs = tokenizer_ko(pred, return_tensors="pt", padding="max_length", max_length=512, truncation=True).input_ids.to(device)
            translated_outputs = model_en_ko.generate(inputs, num_beams=4, max_length=512)
            translated_pred = tokenizer_ko.decode(translated_outputs[0], skip_special_tokens=True)
            preds.append(translated_pred)

    # Step 4: 예측 결과를 sample_submission 형식으로 저장
    df = pd.DataFrame({
        'fname': test_data_ids,
        'summary': preds
    })

    # CSV 파일로 저장
    df.to_csv(output_file_path, index=False)
    print(f"Prediction results saved to {output_file_path}")
    return preds

# test 데이터에 대해 DataLoader 생성 (test.csv 불러오기)
test_df = pd.read_csv('/content/drive/MyDrive/AI_Lab/AI_Lab_online/basic/NLP/Competition/Code/data/test.csv')
test_data_ids = test_df['fname']
test_data = MultiTaskDataset(test_df, tokenizer_ko_en, max_len=512, augment=False, is_test=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=False)

# 추론 및 한국어 요약 결과 저장
output_file_path = '/content/drive/MyDrive/AI_Lab/AI_Lab_online/basic/NLP/Competition/Code/prediction/mBart50_T5_v6_translated.csv'
inference_with_translation(trainer, tokenizer_ko_en, tokenizer_en_ko, test_dataloader, output_file_path, test_data_ids)

# WandB 종료
wandb.finish()
